In [1]:
import pandas as pd
import numpy as np
from scipy import sparse as sps
import nltk
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
from scipy import sparse
from sklearn.preprocessing import normalize
lemmatizer = WordNetLemmatizer() 
words = set(nltk.corpus.words.words())

In [2]:
saved = open("pickle/game_reviews.pickle","rb")
game_reviews = pickle.load(saved)
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637


In [3]:
reviews = game_reviews[['item_id', 'user_id','review']]
reviews = reviews.rename(columns={'user_id': 'user', 'item_id': 'item'})
reviews.shape

(58430, 3)

In [4]:
user_game = open("pickle/user_games.pickle","rb")
user_games = pickle.load(user_game)
user_games.head()

,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat
3,76561197970982479,40,Deathmatch Classic
4,76561197970982479,50,Half-Life: Opposing Force


In [5]:
user_games_list = user_games.rename(columns={'user_id': 'user', 'item_id': 'item'})
user_games_list = user_games_list[['item', 'user']]

user_games_list.head()

,item,user
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479


In [6]:
def process(content):
        
        processed = tokenizer.tokenize(content)

        processed = [token.lower() for token in processed]

        processed = [token for token in processed if token not in stopwords.words('english')]
        
        #processed = [token for token in processed if token in words or not token.isalnum()]
        
        processed = [ps.stem(token) for token in processed]
        
        return processed

In [7]:
item_data = pd.DataFrame({'review': reviews.groupby(['item']).review.apply(lambda x:' '.join(x))})
item_data.reset_index(inplace=True)
item_data['processed_reviews'] = item_data['review'].apply(lambda row: process(row))

In [8]:
item_data.head()

,item,review,processed_reviews
0,10,Cool game Wallbang simulator this game is the ...,"[cool, game, wallbang, simul, game, 1, onlin, ..."
1,20,No Bread. Team Fortress tutkusunun başlangıcı;...,"[bread, team, fortress, tutkusunun, başlangıcı..."
2,30,An excellent war shooter with a great online c...,"[excel, war, shooter, great, onlin, commun, 5,..."
3,40,Best playerbase I have ever met. Everybody is ...,"[best, playerbas, ever, met, everybodi, nice, ..."
4,50,You want a decent Half-Life 1 game that's chea...,"[want, decent, half, life, 1, game, cheap, get..."


In [9]:
#item_data['bow'] = item_data['processed_reviews'].apply(lambda row: ' '.join(row))
#bow = item_data['bow'].tolist()
#len(bow)

In [10]:
bow = item_data['processed_reviews'].tolist()
len(bow)

3682

In [11]:
len(set(bow[0]))

571

In [12]:
from gensim import corpora
dictionary = corpora.Dictionary(bow) 
corpus = [dictionary.doc2bow(text) for text in bow]

In [13]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of articles: %d' % len(corpus))

Number of unique tokens: 70964
Number of articles: 3682


In [14]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel.save('model5.gensim')

In [18]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(ldamodel.print_topics())
doc_lda = ldamodel[corpus]

[(0,
  '0.021*"path" + 0.019*"exil" + 0.013*"plane" + 0.011*"war" + 0.010*"diablo" '
  '+ 0.009*"star" + 0.008*"nope" + 0.006*"gear" + 0.006*"emili" + '
  '0.005*"metal"'),
 (1,
  '0.040*"de" + 0.034*"que" + 0.016*"la" + 0.015*"lo" + 0.013*"juego" + '
  '0.013*"game" + 0.012*"un" + 0.012*"el" + 0.012*"en" + 0.011*"es"'),
 (2,
  '0.016*"spoler" + 0.016*"spolier" + 0.015*"spoiler" + 0.004*"stealth" + '
  '0.002*"runescap" + 0.001*"bottleneck" + 0.001*"cs" + 0.001*"cpu" + '
  '0.001*"rs" + 0.001*"ninja"'),
 (3,
  '0.041*"dlc" + 0.019*"payday" + 0.017*"heist" + 0.015*"updat" + 0.012*"add" '
  '+ 0.012*"new" + 0.012*"overkil" + 0.011*"overpr" + 0.010*"2" + '
  '0.009*"microtransact"'),
 (4,
  '0.040*"ㅤ" + 0.035*"game" + 0.019*"mod" + 0.014*"garri" + 0.011*"fun" + '
  '0.010*"sandbox" + 0.009*"hey" + 0.008*"play" + 0.008*"10" + '
  '0.007*"gamemod"'),
 (5,
  '0.045*"game" + 0.013*"play" + 0.009*"like" + 0.009*"10" + 0.007*"get" + '
  '0.007*"one" + 0.007*"good" + 0.007*"great" + 0.006*"stori